In [1]:
#タイトルより質問を生成・既存質問に追加

#トークン長400超えたときバッチを閉じる

#ペアをdataframeに変換し、csv出力

#妥当な答えかどうかの判定は？

#MTBをずっと読む

In [38]:
def read_docs(file_path):
    f = open(file_path, 'r', encoding='UTF-8')
    data = f.read()
    docs=data.replace(' ','').replace('\n','').replace('\ufeff','').split('。')
    return docs

In [24]:
#トークン長
import MeCab
def getlength(doc):
    doc=doc.replace('-','')
    tagger = MeCab.Tagger('-Owakati') 
    val=tagger.parse(doc).split(' ')
    return len(val)

In [86]:
#バッチ作成　400トークン以下
def batch_docs(docs: list):
    batch_list=[]
    ind=0
    while ind < len(docs):
        batch=[]
        len_sum=0
        while len_sum<300:
            if ind>len(docs)-1:
                break
            batch.append(docs[ind])
            len_sum+=len(docs[ind])
            ind+=1
        batch_list.append('。'.join(batch))
    return batch_list

In [34]:
#スライドバッチ作成　400トークン以下
def slide_docs(docs: list):
    batch_list=[]
    ind=0
    while ind < len(docs):
        batch=docs[ind:ind+4]
        batch_list.append('。'.join(batch))
        ind+=1
    return batch_list

In [79]:
#QA基礎
from transformers import BertJapaneseTokenizer, AutoModelForQuestionAnswering
import torch
model = AutoModelForQuestionAnswering.from_pretrained('output/')  
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 
def suiron(context,question,file=None,limit=False):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    output = model(**inputs)
    answer_start = torch.argmax(output.start_logits)  
    answer_end = torch.argmax(output.end_logits) + 1 
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    #print(answer)
    if 'SEP' in answer:
        return None
    if limit:
        if len(answer.replace(' ',''))>100:
            return None
    return answer.replace(' ','')

In [80]:
#質問計算
def ask(doc,title):
    questions= ["何が発生したか?","事故は何か？","何が起きたか？"]
    ans=[]
    for q in questions:
        tmp=suiron(doc,q,limit=True)
        ans.append(tmp)
    #print(ans)
    if ans[0]==ans[1]:
        truans= ans[0]
    elif ans[1]==ans[2]:
        truans= ans[1]
    elif ans[0]==ans[2]:
        truans= ans[2]
    else:
        truans= None
    result=[title,truans]
    if truans:
        cause=[suiron(doc,title+'の原因は何か？'),suiron(doc,truans+'の原因は何か？')]
    else:
        cause=[suiron(doc,title+'の原因は何か？'),None]
    return [[result[0],cause[0]],[result[1],cause[1]]]

In [83]:
#メイン動作
import tqdm
def extraction(docs,title):
    xs=batch_docs(docs)
    outputs=[]
    for x in tqdm.tqdm(xs):
        out=ask(x,title)
        outputs.extend(out)
    return outputs

# 動作ルール

## 結果を導き出す
### ①　タイトルを採用。
### ②　３つの質問を入力し、二つ以上共通する答えが出れば結果として採用。
### 答えがかなり長文となるものや[SEP]を含む場合は次の質問に支障が出るため除外。

## 原因を聞く
### 上で作成した各々の結果に「の原因は何か？」を追加し、再質問。
### 答えが得られれば確定。

## 文書①　タイトル：朱鷺メッセ連絡デッキ落下事故

In [82]:
import pandas as pd
outputs=extraction(docs,'朱鷺メッセ連絡デッキ落下事故')
pd.DataFrame(outputs,columns=['結果','原因'])

,結果,原因
0,朱鷺メッセ連絡デッキ落下事故,失敗百選
1,None,None
2,朱鷺メッセ連絡デッキ落下事故,なお、当該工区の施工時の請負額は3億4034万5000円である。発注者である新潟県は、設計や...
3,None,None
4,朱鷺メッセ連絡デッキ落下事故,None
5,None,None
6,朱鷺メッセ連絡デッキ落下事故,自重
7,朱鷺メッセ連絡デッキが自重のみによって突然落下した,JSCA
8,朱鷺メッセ連絡デッキ落下事故,斜材ロッド定着部コンクリートの破損
9,None,None


# 文書②　タイトル：富士通 HDD ( ハード・ディスク・ドライブ )不良問題

In [87]:
import pandas as pd
docs=read_docs('sample1.txt')
outputs=extraction(docs,'富士通 HDD ( ハード・ディスク・ドライブ )不良問題')
pd.DataFrame(outputs,columns=['結果','原因'])

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:47<00:00,  2.28s/it]


,結果,原因
0,富士通 HDD ( ハード・ディスク・ドライブ )不良問題,赤リンを含む住友ベークライトのパッケージ封止材
1,パソコンおよびPCサーバ内蔵ハードディスク装置の不具合,赤リンを含む住友ベークライトのパッケージ封止材
2,富士通 HDD ( ハード・ディスク・ドライブ )不良問題,HDD
3,None,None
4,富士通 HDD ( ハード・ディスク・ドライブ )不良問題,HDDコントローラLSI
5,None,None
6,富士通 HDD ( ハード・ディスク・ドライブ )不良問題,パッケージ封止材
7,パッケージ封止材に起因する問題,CALCE
8,富士通 HDD ( ハード・ディスク・ドライブ )不良問題,封止材
9,トラブル,LSI


# 文書③　タイトル：JCO ウラン加工工場での臨界事故 

In [89]:
import pandas as pd
docs=read_docs('sample2.txt')
outputs=extraction(docs,'JCO ウラン加工工場での臨界事故')
pd.DataFrame(outputs,columns=['結果','原因'])

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:55<00:00,  2.41s/it]


,結果,原因
0,JCO ウラン加工工場での臨界事故,JCO
1,臨界事故,JCO
2,JCO ウラン加工工場での臨界事故,本来、使用目的が異なり、また、臨界安全形状に設計されていない沈殿槽に、臨界質量以上のウランを...
3,事故,本来、使用目的が異なり、また、臨界安全形状に設計されていない沈殿槽に、臨界質量以上のウランを...
4,JCO ウラン加工工場での臨界事故,核燃料工場
5,事故,核燃料工場
6,JCO ウラン加工工場での臨界事故,金属ナトリウムの漏洩事故
7,None,None
8,JCO ウラン加工工場での臨界事故,"その時大きなエネルギーを発生すると同時に、原子核の崩壊毎に2,3個の中性子が発生する。発生し..."
9,None,None


# 課題
* 多数が不正解と思われる
* 出力ペアが本当に因果関係か確かめる必要がある。